**Definizione delle funzioni di identificazione dei bordi e di crop+resize**

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def load_image(file):
    img = Image.open(file)
    img = img.convert('L')
    img = np.array(img)
    return img

def save_image(img, file):
    img = Image.fromarray(img, mode='L')
    img.save(file)

import os
import numpy as np
from tqdm import tqdm
from PIL import Image

def findBorders(adc):
    # FIND BLACK BORDERS IN T2W CHANNEL
    # Make t2w boolean
    canale_bw = adc > 0
    # Find non-black rows and columns
    rows = np.any(canale_bw, axis=1)
    cols = np.any(canale_bw, axis=0)
    # Get first and last non-black row and column indexes
    rmin, rmax = np.where(rows)[0][[0, -1]] if np.any(rows) else (0, adc.shape[0])
    cmin, cmax = np.where(cols)[0][[0, -1]] if np.any(cols) else (0, adc.shape[1])
    return rmin, rmax, cmin, cmax

def cropResizeImages(adc, hbv, t2w, mask):
    # CROP IMAGES
    # Get original image size
    original_shape = t2w.shape
    # Find black borders in t2w channel
    rmin_adc, rmax_adc, cmin_adc, cmax_adc = findBorders(adc)
    # Get final cropped image size
    cropDimension = min(rmax_adc - rmin_adc, cmax_adc - cmin_adc)
    if cropDimension % 2 != 0: # ensure crop_size is even
        cropDimension -= 1
    # La parte sotto è commentata perchè in questo caso il crop deve essere sempre fatto (e verrà sicuramente fatto)
    
    # if abs(t2w.shape[0] - cropDimension) < 5: # do not crop if the black region is too small
    #     cropDimension = t2w.shape[0]
    #     flagCrop = False
    #     # resize images to 256x256
    #     resizeShape = t2w.shape
    #     adc_resized = np.array(Image.fromarray(adc, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    #     hbv_resized = np.array(Image.fromarray(hbv, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    #     t2w_resized = np.array(Image.fromarray(t2w, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    #     mask_resized = np.array(Image.fromarray(mask, mode='L').resize(resizeShape, Image.NEAREST), dtype=np.uint8)
    #     # Create the dictionary to store resize metadata
    #     cropMetadata = {'flagCrop': flagCrop, 'original_shape': original_shape, 'resizeShape': resizeShape}
    #     return adc_resized, hbv_resized, t2w_resized, cropMetadata
    # else:
    #     flagCrop = True
    flagCrop = True
    # Get center of the non-black region
    center_x = (cmin_adc + cmax_adc) // 2
    center_y = (rmin_adc + rmax_adc) // 2
    # Get start and end coordinates for cropping
    start_x = max(center_x - cropDimension // 2, 0)
    start_y = max(center_y - cropDimension // 2, 0)
    end_x = min(start_x + cropDimension, adc.shape[1])
    end_y = min(start_y + cropDimension, adc.shape[0])

    # Crop images at the center of the non-black region
    adc_cropped = adc[start_y:end_y, start_x:end_x]
    hbv_cropped = hbv[start_y:end_y, start_x:end_x]
    t2w_cropped = t2w[start_y:end_y, start_x:end_x]
    mask = mask[start_y:end_y, start_x:end_x]

    cropped_shape = t2w_cropped.shape

    # RESIZE IMAGES
    resizeShape = t2w.shape
    adc_cropped_resized = np.array(Image.fromarray(adc_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    hbv_cropped_resized = np.array(Image.fromarray(hbv_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    t2w_cropped_resized = np.array(Image.fromarray(t2w_cropped, mode='L').resize(resizeShape, Image.LANCZOS), dtype=np.uint8)
    mask_cropped_resized = np.array(Image.fromarray(mask, mode='L').resize(resizeShape, Image.NEAREST), dtype=np.uint8)
    # Create a dictionary to store crop metadata
    # cropMetadata = {'start_x': start_x, 'start_y': start_y, 'end_x': end_x, 'end_y': end_y, 'cropDimension': cropDimension, 'cropped_shape': cropped_shape, 'center_x': center_x, 'center_y': center_y, 'original_shape': original_shape, 'resizeShape': resizeShape, 'flagCrop': flagCrop}

    return adc_cropped_resized, hbv_cropped_resized, mask_cropped_resized



**Applicazione del crop ai pazienti rientranti nella casistica 3**

In [ ]:
# Path in cui sono presenti le cartelle dei pazienti
pathDataset = os.path.join(".", "PAZIENTI CASISTICA 3")

# Loop sui pazienti della casistica 3
for paziente in tqdm(os.listdir(pathDataset)):
    pathPaziente = os.path.join(pathDataset, paziente)
    # Loop sulle slice di un paziente della casistica 2
    for slice in os.listdir(os.path.join(pathPaziente, "adc")):
        # Load delle immagini
        adc = load_image(os.path.join(pathPaziente, "adc", slice))
        hbv = load_image(os.path.join(pathPaziente, "hbv", slice))
        t2w = load_image(os.path.join(pathPaziente, "t2w", slice))
        mask = load_image(os.path.join(pathPaziente, "manual", slice))
        # Crop e resize delle immagini
        adc_cropped_resized, hbv_cropped_resized, mask_cropped_resized = cropResizeImages(adc, hbv, t2w, mask)
        # Creazione dell'immagine stacked
        stacked = np.stack([adc_cropped_resized, hbv_cropped_resized, t2w], axis=-1)
        # Salvataggio delle nuove immagini
        save_image(adc_cropped_resized, os.path.join(pathPaziente, "adc", slice))
        save_image(hbv_cropped_resized, os.path.join(pathPaziente, "hbv", slice))
        save_image(mask_cropped_resized, os.path.join(pathPaziente, "manual", slice))
        stacked = Image.fromarray(stacked, mode='RGB')
        stacked.save(os.path.join(pathPaziente, "stacked", slice))